In [1]:
from langchain.agents import create_agent #for creating the agent
from langchain_community.agent_toolkits.sql.toolkit import SQLDatabaseToolkit #for SQL database toolkit
from langchain_community.utilities.sql_database import SQLDatabase
from langchain_core.prompts import ChatPromptTemplate #for creating chat prompts templates
from langchain_google_genai import ChatGoogleGenerativeAI #for using Google Gemini LLM API
from dotenv import load_dotenv
import os

load_dotenv('../../credentials.env') #load environment variables (credentials and API keys) from a .env file


True

In [2]:
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash-preview-09-2025",
    google_api_key=os.getenv("GOOGLE_API_KEY"),
    
    )

In [ ]:
# for running locally
# from langchain_openai import ChatOpenAI

# llm = ChatOpenAI(
#     base_url="http://127.0.0.1:1234/v1",
#     model="qwen/qwen3-4b-2507",
#     api_key="f"
# )

In [3]:
system_prompt = """You are a SQL expert assistant for an expense tracking app. Follow these instructions:
- Always verify table schemas before querying
- Use proper MySQL syntax
- Format results clearly and concisely
- If no data exists, state that explicitly
- Double-check date ranges in queries
- Provide short, actionable insights, tips, or budget suggestions when relevant
- Keep explanations brief and focused on practical guidance"""


In [4]:
mysql_uri = f'mysql+mysqlconnector://{os.getenv("MYSQL_USERNAME")}:{os.getenv("MYSQL_PASSWORD")}@{os.getenv("MYSQL_HOST")}:{os.getenv("MYSQL_PORT")}/{os.getenv("DATABASE_NAME")}'
db = SQLDatabase.from_uri(mysql_uri) # type: ignore

toolkit = SQLDatabaseToolkit(db=db, llm=llm) # setting up the SQL toolkit
agent = create_agent(llm, toolkit.get_tools(), system_prompt=system_prompt) # creating the agent

In [5]:
def invoke_agent(user_query):

    response = agent.invoke({
        "messages": [
            {"role": "user", "content": str(user_query)}
        ]
})
    
    return response


In [7]:
response = invoke_agent("what is the most categary that i spent the most on?")
print(response)

{'messages': [HumanMessage(content='what is the most categary that i spent the most on?', additional_kwargs={}, response_metadata={}, id='48618a02-1cf9-4009-afea-70881fc9e749'), AIMessage(content='', additional_kwargs={'function_call': {'name': 'sql_db_list_tables', 'arguments': '{"tool_input": ""}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash-preview-09-2025', 'safety_ratings': [], 'grounding_metadata': {}, 'model_provider': 'google_genai'}, id='lc_run--b6f5c0c2-1f70-4783-a39e-8921489fd49a-0', tool_calls=[{'name': 'sql_db_list_tables', 'args': {'tool_input': ''}, 'id': '5f62a26e-1058-4549-9d62-c7548a039838', 'type': 'tool_call'}], usage_metadata={'input_tokens': 474, 'output_tokens': 109, 'total_tokens': 583, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 86}}), ToolMessage(content='categories, expenses, users', name='sql_db_list_tables', id='217ec6ef-f53c-4

In [ ]:
# print(response['messages'][-1].content)

The category you spent the most on is **Shopping**, with a total of **45.00**.

**Tip:** Since Shopping is your top expense, consider setting a specific monthly budget for this category to help control your spending.
